#  **Testbench for experimenting on 2D-CNN pipeline for Classification - PyTorch**



**Please MAKE sure that you are using GPU . Go to Runtime>> Change Runtime type  and select GPU**

#  **Connect to Google Drive**
 Allow Google Colab to connect to your GDrive

In [193]:
from google.colab import drive
drive.mount('/content/gdrive')

ModuleNotFoundError: No module named 'google.colab'

Change the current directory to Virtual Machine (VM)

In [ ]:
%cd '/content/'

/content


List all files in the directory

In [ ]:
!ls

gdrive	sample_data


# **Check GPU Status**
Check the available GPU from the Google Server, GPU model and other related information.

In [1]:
import torch
print("Is CUDA enabled GPU Available?", torch.cuda.is_available())
print("Is GPU Initialized yet?", torch.cuda.is_initialized())
print("GPU Number:", torch.cuda.device_count())
print("Current GPU Index:", torch.cuda.current_device())
print("GPU Type:", torch.cuda.get_device_name(device=None))
print("GPU Capability:", torch.cuda.get_device_capability(device=None))

Is CUDA enabled GPU Available? False
Is GPU Initialized yet? False
GPU Number: 0


AssertionError: Torch not compiled with CUDA enabled

# **Import and Prepare Code and Dataset files** 
1- Copy dataset file and codes to your Google Colab instance (Change file names if necessary ), dont have to copy the data files to the content here, as it will be copied in the next code

1- Copy the Data to the Virtual Machine

2- Unzip the dataset file in the instance,
   Please rename the downloaded file as Data.zip if it has a different name

In [ ]:
!pip3 install --upgrade gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


Import Codes

In [ ]:
%cd '/content/'
!gdown --id 1x4s7escuDphytkQhIwWRHcmnBNwEUkXl

/content
/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1x4s7escuDphytkQhIwWRHcmnBNwEUkXl
To: /content/codes.zip
100% 583M/583M [00:05<00:00, 103MB/s]


In [ ]:
!jar xvf codes.zip

 inflated: chexnet.pth.tar
 inflated: chexnet.py
 inflated: config.py
 inflated: config_inference.py
 inflated: config_OutputProbabilities.py
 inflated: darknet53.pth.tar
 inflated: darknet53.py
 inflated: inceptionresnetv2.pth
 inflated: Inference.py
 inflated: InferenceTime.py
 inflated: models.py
 inflated: Output_Probability.py
 inflated: README.txt
 inflated: selfonn.py
 inflated: testbench.py
 inflated: train.py
 inflated: utils.py
 inflated: xception.py


Run this code block if the ZIP extracts into a sub-folder instead of the root dir

In [ ]:
import shutil
import os
    
source_dir = '/content/codes'
target_dir = '/content/'
    
file_names = os.listdir(source_dir)
    
for file_name in file_names:
    shutil.move(os.path.join(source_dir, file_name), target_dir)

Import Data

In [ ]:
%cd '/content/'
!gdown --id 1UJLb-oRfZ4yUbP-6RkFaq1mGOXX4Dw6N
!jar xvf Data.zip

# **Install and Import Required Libraries**
Use 'pip' to install libraries. Put '!' i.e. Exclamation Mark before each command, exclusive to Jupyter Notebooks. Some are already installed by Google Colab and others are required. If the IMPORT statements are present in the python code being run, no need to follow this step.

Install or Update Libraries

In [3]:
# !pip install torch==1.11.0 torchvision==0.12.0 torchaudio
# !pip install torchvision
# !pip install torchsummary
# !pip install numpy
# !pip install pandas
!pip install scikit-learn
!pip install scikit-image
!pip install ipython
!pip install Pillow
!pip install matplotlib
!pip install seaborn
!pip install gdown
!pip install efficientnet-pytorch
!pip install timm
!pip install tqdm
!pip install torchviz
!pip install graphviz

  Using cached efficientnet_pytorch-0.7.1.tar.gz (21 kB)
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=bfe8754a6ce7ebcc21342a81321dc4423766db4f0902642e25fbd719aebe9f11
  Stored in directory: c:\users\sakib mahmud\appdata\local\pip\cache\wheels\29\16\24\752e89d88d333af39a288421e64d613b5f652918e39ef1f8e3
Successfully built efficientnet-pytorch
  Using cached torchviz-0.0.2.tar.gz (4.9 kB)
  Using cached graphviz-0.20.1-py3-none-any.whl (47 kB)
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4150 sha256=4f77172c81d9838f5b6a3d87887a2b58c79d8cedf2c0f65a265ea77df05d8a8b
  Stored in directory: c:\users\sakib mahmud\appdata\local\pip\cache\wheels\29\65\6e\db2515eb1dc760fecd36b40d54df65c1e18534013f1c037e2e
Successfully built torchviz


Import Libraries

In [4]:
# Printing out all outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
# PyTorch
import torch
import torch.nn as nn
from torch import optim, cuda, tensor
from torch.utils.data import DataLoader
from torchvision import transforms, models
from torchvision.models import densenet201, DenseNet201_Weights
from torchvision.models.feature_extraction import get_graph_node_names, create_feature_extractor
# warnings
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
# Data science tools
import os
import timm
import numpy as np
from os import path
from importlib import import_module
# Visualizations
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 14

# **MAIN**



Print Pipeline options

In [ ]:
f = open("README.txt", "r")
print(f.read())

Load Model

In [ ]:
model_file = torch.load('C:/Users/Sakib Mahmud/Documents/VS Code/2D_Feature_Extractor_PyTorch/Trained_Models/DenseNet201_Tumor_Classification_Fold_1.pt')
print(model_file.keys())
model = models.densenet201(pretrained=True)
print(model)

In [5]:
from SelfONN import SuperONN1d, SuperONN2d, SelfONN1d, SelfONN2d

ModuleNotFoundError: No module named 'SelfONN'

In [7]:
class CNN_Classifier(nn.Module):
    def __init__(self, in_channels, class_num, final_activation):
        super().__init__()
        if final_activation == 'ELU':
            self.classifier = nn.Sequential(
                nn.Linear(in_channels, class_num),
                nn.ELU(alpha=1.0, inplace=False)
                )
        elif final_activation == 'Hardshrink':
            self.classifier = nn.Sequential(
                nn.Linear(in_channels, class_num),
                nn.Hardshrink(lambd=0.5)
                )
        elif final_activation == 'Hardsigmoid':
            self.classifier = nn.Sequential(
                nn.Linear(in_channels, class_num),
                nn.Hardsigmoid(inplace=False)
                )
        elif final_activation == 'Hardtanh':
            self.classifier = nn.Sequential(
                nn.Linear(in_channels, class_num),
                nn.Hardtanh(min_val= 0.0, max_val=1.0, inplace=False, min_value=None, max_value=None)
                )
        elif final_activation == 'Hardswish':
            self.classifier = nn.Sequential(
                nn.Linear(in_channels, class_num),
                nn.Hardswish(inplace=False)
                )
        elif final_activation == 'Hardtanh':
            self.classifier = nn.Sequential(
                nn.Linear(in_channels, class_num),
                nn.Hardtanh(min_val=-1.0, max_val=1.0, inplace=False, min_value=None, max_value=None)
                )
        elif final_activation == 'LeakyReLU':
            self.classifier = nn.Sequential(
                nn.Linear(in_channels, class_num),
                nn.LeakyReLU(negative_slope=0.01, inplace=False)
                )
        elif final_activation == 'LogSigmoid':
            self.classifier = nn.Sequential(
                nn.Linear(in_channels, class_num),
                nn.LogSigmoid()
                )
        elif final_activation == 'LogSoftmax':
            self.classifier = nn.Sequential(
                UnSqueezeLayer(),
                nn.Tanh(),
                SelfONN2d(in_channels=in_channels, out_channels=class_num, kernel_size=1, q=1, mode='fast', dropout=None),
                nn.Tanh(),
                SqueezeLayer(),
                nn.LogSoftmax(dim=1) 
            )
        elif final_activation == 'PReLU':
            self.classifier = nn.Sequential(
                nn.Linear(in_channels, class_num),
                nn.PReLU(num_parameters=1, init=0.25, device=None, dtype=None)
                )
        elif final_activation == 'ReLU':
            self.classifier = nn.Sequential(
                nn.Linear(in_channels, class_num),
                nn.ReLU(inplace=False)
                )
        elif final_activation == 'ReLU6':
            self.classifier = nn.Sequential(
                nn.Linear(in_channels, class_num),
                nn.ReLU6(inplace=False)
                )
        elif final_activation == 'RReLU':
            self.classifier = nn.Sequential(
                nn.Linear(in_channels, class_num),
                nn.RReLU(lower=0.125, upper=0.3333333333333333, inplace=False)
                )
        elif final_activation == 'SELU':
            self.classifier = nn.Sequential(
                nn.Linear(in_channels, class_num),
                nn.SELU(inplace=False)
                )
        elif final_activation == 'CELU':
            self.classifier = nn.Sequential(
                nn.Linear(in_channels, class_num),
                nn.CELU(alpha=1.0, inplace=False)
                )
        elif final_activation == 'GELU':
            self.classifier = nn.Sequential(
                nn.Linear(in_channels, class_num),
                nn.GELU(approximate='none')
                )
        elif final_activation == 'Sigmoid':
            self.classifier = nn.Sequential(
                nn.Linear(in_channels, class_num),
                nn.Sigmoid()
                )
        elif final_activation == 'SiLU':
            self.classifier = nn.Sequential(
                nn.Linear(in_channels, class_num),
                nn.SiLU(inplace=False)
                )
        elif final_activation == 'Softmin':
            self.classifier = nn.Sequential(
                nn.Linear(in_channels, class_num),
                nn.Softmin(dim=None)
                )
        elif final_activation == 'Softmax':
            self.classifier = nn.Sequential(
                nn.Linear(in_channels, class_num),
                nn.Softmax(dim=None)
                )
        elif final_activation == 'Softmax2d':
            self.classifier = nn.Sequential(
                nn.Linear(in_channels, class_num),
                nn.Softmax2d()
                )
        elif final_activation == 'Softplus':
            self.classifier = nn.Sequential(
                nn.Linear(in_channels, class_num),
                nn.Softplus(beta=1, threshold=20)
                )
        elif final_activation == 'Softshrink':
            self.classifier = nn.Sequential(
                nn.Linear(in_channels, class_num),
                nn.Softshrink(lambd=0.5)
                )
        elif final_activation == 'Softsign':
            self.classifier = nn.Sequential(
                nn.Linear(in_channels, class_num),
                nn.Softsign()
                )
        elif final_activation == 'Tanh':
            self.classifier = nn.Sequential(
                nn.Linear(in_channels, class_num),
                nn.Tanh()
                )
        elif final_activation == 'Tanhshrink':
            self.classifier = nn.Sequential(
                nn.Linear(in_channels, class_num),
                nn.Tanhshrink()
                )
        elif final_activation == 'GLU':
            self.classifier = nn.Sequential(
                nn.Linear(in_channels, class_num),
                nn.GLU(dim=-1)
                )
        torch.nn.init.xavier_uniform_(self.classifier[0].weight)
        self.classifier[0].bias.data.fill_(0.01) 
     
    def forward(self,x):
        x = self.classifier(x)
        return x

In [29]:
class SqueezeLayer(nn.Module):
    def forward(self,x):
        x = x.squeeze(2)
        x = x.squeeze(2)
        return x 

In [30]:
class UnSqueezeLayer(nn.Module):
    def forward(self,x):
        x = x.unsqueeze(2).unsqueeze(3)
        return x

In [ ]:
model = models.regnet_y_400mf(weights='IMAGENET1K_V1')
num_ftrs = model.fc.in_features
model.fc = CNN_Classifier(in_channels=num_ftrs, class_num=2, final_activation='LogSigmoid')
x = torch.rand(8, 3, 224, 224)
y = model(x)
target = torch.Tensor([1,0,0,0,0,1,1,1])
print(y.size())
print(target.size())
criterion = nn.NLLLoss()
loss = criterion(y,target)

Extract single targeted layers from pretrained models

In [ ]:
extracted_layer_1 = model.features.transition1.conv
extracted_layer_2 = model.features.transition2.conv
extracted_layer_3 = model.features.transition3.conv
extracted_layer_4 = model.features.denseblock4.denselayer32.conv1
print(extracted_layer_1)
print(extracted_layer_2)
print(extracted_layer_3)
print(extracted_layer_4)

Extract a chunk of pretrained models

In [ ]:
model = models.inception_v3(weights='DEFAULT')
Conv0 = nn.Sequential(*list(model.children())[0:])
Conv1 = nn.Sequential(*list(model.children())[10:10])
Conv2 = nn.Sequential(*list(model.children())[0:15])
Conv3 = nn.Sequential(*list(model.children())[0:15],*list(model.children())[16:19])
print(model)

In [ ]:
model = models.mobilenet_v2(weights='IMAGENET1K_V2')
Conv1 = nn.Sequential(model.features)
print(Conv1)

In [ ]:
model = models.densenet201(weights='DEFAULT')
Conv1 = nn.Sequential(*list(model.features.children())[0:5])
Conv2 = nn.Sequential(*list(model.features.children())[0:7])
Conv3 = nn.Sequential(*list(model.features.children())[0:9])
Conv4 = nn.Sequential(*list(model.features.children())[0:])
print(Conv2)


In [88]:
class PHNet(nn.Module):
    def __init__(self, class_num):
        super(PHNet, self).__init__()
        model1 = models.densenet201(weights='DEFAULT')
        model2 = models.inception_v3(weights='DEFAULT')
        self.ExtractedDenseBlock1 = nn.Sequential(*list(model1.features.children())[0:7])
        self.ExtractedDenseBlock2 = nn.Sequential(*list(model1.features.children())[0:9])
        self.ExtractedDenseBlock3 = nn.Sequential(*list(model1.features.children())[0:])
        self.ExtractedInceptionBlocks1 = nn.Sequential(*list(model2.children())[0:10])
        self.ExtractedInceptionBlocks2 = nn.Sequential(*list(model2.children())[0:15])
        self.ExtractedInceptionBlocks3 = nn.Sequential(*list(model2.children())[0:15],*list(model2.children())[16:19])
        self.zeropad1 = nn.ZeroPad2d((1, 1, 1, 1))
        self.zeropad2 = nn.ZeroPad2d((1, 0, 1, 0))
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')
        self.aux1 = nn.Sequential(
            nn.AvgPool2d(kernel_size=5, stride=3),
            nn.Conv2d(800, 50, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True),
            nn.ReLU6(),
            nn.Conv2d(50, 800, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(800, eps=1e-05, momentum=0.1, affine=True),
            nn.ReLU6(),
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten(),
            nn.Linear(800, class_num),
            nn.LogSoftmax()
            )
        self.aux2 = nn.Sequential(
            nn.AvgPool2d(kernel_size=3, stride=3),
            nn.Conv2d(2560, 320, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(320, eps=1e-05, momentum=0.1, affine=True),
            nn.ReLU6(),
            nn.Conv2d(320, 2560, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm2d(2560, eps=1e-05, momentum=0.1, affine=True),
            nn.ReLU6(),
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten(),
            nn.Linear(2560, class_num),
            nn.LogSoftmax()
            )
        self.final_classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Dropout(p=0.2),
            nn.Flatten(),
            nn.Linear(3968, class_num),
            nn.LogSoftmax()
            )
    
    def forward(self, x):
        Dout1 = self.ExtractedDenseBlock1(x)
        Dout2 = self.ExtractedDenseBlock2(x)
        Dout3 = self.ExtractedDenseBlock3(x)
        Iout1 = self.upsample(self.zeropad1(torch.max_pool2d(self.ExtractedInceptionBlocks1(x),2)))
        Iout2 = self.upsample(self.zeropad2(torch.max_pool2d(self.ExtractedInceptionBlocks2(x),2)))
        Iout3 = self.zeropad1(self.ExtractedInceptionBlocks3(x))
        out1 = torch.cat((Dout1, Iout1), dim=1)
        out2 = torch.cat((Dout2, Iout2), dim=1)
        out3 = torch.cat((Dout3, Iout3), dim=1)
        final_out1 = self.aux1(out1)
        final_out2 = self.aux2(out2)
        final_out3 = self.final_classifier(out3)
        return final_out3, final_out2, final_out1

In [89]:
import torch

N, C, H, W = 10, 3, 224, 224
x = torch.empty(N, C, H, W)
# y = x.to(memory_format=torch.channels_last)
# print(x.size())
# print(x.stride())  # Ouputs: (3072, 1024, 32, 1)

y = PHNet(3)(x)

torch.Size([10, 800, 28, 28])


c:\Users\Sakib Mahmud\anaconda3\lib\site-packages\torch\nn\modules\container.py:139: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Modfied DenseNet201 Network

In [73]:
class Modified_DenseNet201(nn.Module):
    def __init__(self, class_num):
        super(Modified_DenseNet201, self).__init__()
        model = models.densenet201(weights='IMAGENET1K_V1')
        self.ExtractedDenseBlock1 = nn.Sequential(*list(model.features.children())[0:6])
        self.ExtractedDenseBlock2 = nn.Sequential(*list(model.features.children())[0:8])
        self.ExtractedDenseBlock3 = nn.Sequential(*list(model.features.children())[0:10])
        self.ExtractedDenseBlock4 = nn.Sequential(*list(model.features.children())[0:])
        self.downSamp1 = nn.MaxPool2d(4)
        self.downSamp2 = nn.MaxPool2d(2)
        self.conv1 = nn.Sequential(nn.Conv2d(3200, 160, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True))
        self.classifier = nn.Sequential(
            nn.Flatten(start_dim=1),
            nn.Linear(160*7*7, class_num),
            nn.LogSoftmax(dim=None)
            )
    
    def forward(self, x):
        out1 = self.ExtractedDenseBlock1(x)
        out2 = self.ExtractedDenseBlock2(x)
        out3 = self.ExtractedDenseBlock3(x)
        out4 = self.ExtractedDenseBlock4(x)
        out1 = self.downSamp1(out1)
        out2 = self.downSamp2(out2)
        out_cat = torch.cat((out1,out2,out3,out4), dim=1)
        out = self.conv1(out_cat)
        out = self.classifier(out)
        return out

Modified DenseNet201 Model with multiple Auxilliary Losses and Multiheaded Classifier

In [9]:
class Modified_DenseNet201_AUX(nn.Module):
    def __init__(self, class_num):
        super(Modified_DenseNet201_AUX, self).__init__()
        model = models.densenet201(weights='IMAGENET1K_V1')
        self.ExtractedDenseBlock1 = nn.Sequential(*list(model.features.children())[0:5])
        self.ExtractedDenseBlock2 = nn.Sequential(*list(model.features.children())[0:7])
        self.ExtractedDenseBlock3 = nn.Sequential(*list(model.features.children())[0:9])
        self.ExtractedDenseBlock4 = nn.Sequential(*list(model.features.children())[0:])
        self.downSamp1 = nn.MaxPool2d(8)
        self.downSamp2 = nn.MaxPool2d(4)
        self.downSamp3 = nn.MaxPool2d(2)
        self.aux1 = nn.Sequential(
            nn.AvgPool2d(kernel_size=7, stride=3),
            nn.Conv2d(256, 64, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True),
            nn.ReLU6(),
            nn.Conv2d(64, 256, kernel_size=7, stride=1, padding=0),
            nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True),
            nn.ReLU6(),
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten(),
            nn.Linear(256, class_num),
            nn.LogSoftmax()
            )
        self.aux2 = nn.Sequential(
            nn.AvgPool2d(kernel_size=5, stride=3),
            nn.Conv2d(512, 128, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True),
            nn.ReLU6(),
            nn.Conv2d(128, 512, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True),
            nn.ReLU6(),
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten(),
            nn.Linear(512, class_num),
            nn.LogSoftmax()
            )
        self.aux3 = nn.Sequential(
            nn.AvgPool2d(kernel_size=3, stride=3),
            nn.Conv2d(1792, 224, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(224, eps=1e-05, momentum=0.1, affine=True),
            nn.ReLU6(),
            nn.Conv2d(224, 1792, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm2d(1792, eps=1e-05, momentum=0.1, affine=True),
            nn.ReLU6(),
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten(),
            nn.Linear(1792, class_num),
            nn.LogSoftmax()
            )
        self.final_classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Dropout(p=0.2),
            nn.Flatten(),
            nn.Linear(4480, class_num),
            nn.LogSoftmax()
            )
    
    def forward(self, x):
        out1 = self.ExtractedDenseBlock1(x)
        out2 = self.ExtractedDenseBlock2(x)
        out3 = self.ExtractedDenseBlock3(x)
        out4 = self.ExtractedDenseBlock4(x)
        out1_1 = self.downSamp1(out1)
        out2_1 = self.downSamp2(out2)
        out3_1 = self.downSamp3(out3)
        out_cat = torch.cat((out1_1, out2_1, out3_1, out4), dim=1)
        final_out1 = self.aux1(out1)
        final_out2 = self.aux2(out2)
        final_out3 = self.aux3(out3)
        final_out4 = self.final_classifier(out_cat)
        return final_out4, final_out3, final_out2, final_out1

Modified InceptionV3 model a multiheaded classifier

In [ ]:
class Modified_InceptionV3(nn.Module):
    def __init__(self, class_num):
        super(Modified_InceptionV3, self).__init__()
        model = models.inception_v3(weights='IMAGENET1K_V1')
        self.ExtractedInceptionBlocks1 = nn.Sequential(*list(model.children())[0:10])
        self.ExtractedInceptionBlocks2 = nn.Sequential(*list(model.children())[0:15])
        self.ExtractedInceptionBlocks3 = nn.Sequential(*list(model.children())[0:15],*list(model.children())[16:19])
        self.downSamp1 = nn.MaxPool2d(4)
        self.downSamp2 = nn.MaxPool2d(2)
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Dropout(p=0.2),
            nn.Flatten(),
            nn.Linear(3104, class_num),
            nn.LogSoftmax()
            )
    
    def forward(self, x):
        out1 = self.ExtractedInceptionBlocks1(x)
        out2 = self.ExtractedInceptionBlocks2(x)
        out3 = self.ExtractedInceptionBlocks3(x)
        out1 = self.downSamp1(out1)
        out2 = self.downSamp2(out2)
        out_cat = torch.cat((out1,out2,out3), dim=1)
        out = self.classifier(out_cat)
        return out

Modified InceptionV3 model with a Multiheaded classifier and Auxilliary Losses

In [19]:
class Modified_InceptionV3_AUX(nn.Module):
    def __init__(self, class_num):
        super(Modified_InceptionV3_AUX, self).__init__()
        model = models.inception_v3(weights='IMAGENET1K_V1')
        self.ExtractedInceptionBlocks1 = nn.Sequential(*list(model.children())[0:10])
        self.ExtractedInceptionBlocks2 = nn.Sequential(*list(model.children())[0:15])
        self.ExtractedInceptionBlocks3 = nn.Sequential(*list(model.children())[0:15],*list(model.children())[16:19])
        self.downSamp1 = nn.MaxPool2d(4)
        self.downSamp2 = nn.MaxPool2d(2)
        self.aux1 = nn.Sequential(
            nn.AvgPool2d(kernel_size=7, stride=3),
            nn.Conv2d(288, 72, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(72, eps=1e-03, momentum=0.1, affine=True),
            nn.ReLU6(),
            nn.Conv2d(72, 288, kernel_size=7, stride=1, padding=0),
            nn.BatchNorm2d(288, eps=1e-03, momentum=0.1, affine=True),
            nn.ReLU6(),
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten(),
            nn.Linear(288, class_num),
            nn.LogSoftmax()
            )
        self.aux2 = nn.Sequential(
            nn.AvgPool2d(kernel_size=5, stride=3),
            nn.Conv2d(768, 128, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(128, eps=1e-03, momentum=0.1, affine=True),
            nn.ReLU6(),
            nn.Conv2d(128, 768, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(768, eps=1e-03, momentum=0.1, affine=True),
            nn.ReLU6(),
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten(),
            nn.Linear(768, class_num),
            nn.LogSoftmax()
            )
        self.final_classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Dropout(p=0.2),
            nn.Flatten(),
            nn.Linear(3104, class_num),
            nn.LogSoftmax()
            )
    
    def forward(self, x):
        out1 = self.ExtractedInceptionBlocks1(x)
        out2 = self.ExtractedInceptionBlocks2(x)
        out3 = self.ExtractedInceptionBlocks3(x)
        out1_1 = self.downSamp1(out1)
        out2_1 = self.downSamp2(out2)
        out_cat = torch.cat((out1_1,out2_1,out3), dim=1)
        final_out1 = self.aux1(out1)
        final_out2 = self.aux2(out2)
        final_out3 = self.final_classifier(out_cat)
        return final_out3, final_out2, final_out1

DenseNet201 + InceptionV3 with Auxilliary Losses= DenseInception_AUX Model

In [ ]:
class DenseInception_AUX(nn.Module):
    def __init__(self, class_num):
        super(DenseInception_AUX, self).__init__()
        model_densenet201 = models.densenet201(weights='IMAGENET1K_V1')
        model_inceptionv3 = models.inception_v3(weights='IMAGENET1K_V1')
        self.ExtractedDenseBlock = nn.Sequential(
          *list(model_densenet201.features.children()),
          nn.MaxPool2d(2, stride=None, padding=0, dilation=1)
          )
        self.ExtractedInceptionBlock = nn.Sequential(
          *list(model_inceptionv3.children())[0:15],*list(model_inceptionv3.children())[16:19],
          nn.MaxPool2d(2, stride=None, padding=0, dilation=1)
          )
        self.final_classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Dropout(p=0.2),
            nn.Flatten(),
            nn.Linear(3968, class_num),
            nn.LogSoftmax()
            )
        self.aux1 = nn.Sequential(
            nn.AvgPool2d(kernel_size=5, stride=3),
            nn.Conv2d(1920, 240, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(240, eps=1e-05, momentum=0.1, affine=True),
            nn.ReLU6(),
            nn.Conv2d(240, 1920, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm2d(1920, eps=1e-05, momentum=0.1, affine=True),
            nn.ReLU6(),
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten(),
            nn.Linear(1920, class_num),
            nn.LogSoftmax()
            )
        self.aux2 = nn.Sequential(
            nn.AvgPool2d(kernel_size=5, stride=3),
            nn.Conv2d(2048, 256, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(256, eps=1e-03, momentum=0.1, affine=True),
            nn.ReLU6(),
            nn.Conv2d(256, 2048, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm2d(2048, eps=1e-03, momentum=0.1, affine=True),
            nn.ReLU6(),
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten(),
            nn.Linear(2048, class_num),
            nn.LogSoftmax()
            )
    
    def forward(self, x):
        out_densenet201 = self.ExtractedDenseBlock(x)
        out_inceptionv3 = self.ExtractedInceptionBlock(x)
        out_cat = torch.cat((out_densenet201, out_inceptionv3), dim=1)
        final_out1 = self.final_classifier(out_cat)
        final_out2 = self.aux1(out_densenet201)
        final_out3 = self.aux2(out_inceptionv3)
        return final_out1, final_out2, final_out3

In [13]:
import torch

N, C, H, W = 10, 32, 32, 3
x = torch.empty(N, C, H, W)
y = x.to(memory_format=torch.channels_last)
print(x.size())
print(y.size())
print(x.stride())  # Ouputs: (3072, 1024, 32, 1)
print(y.stride())  # Ouputs: (3072, 1024, 32, 1)

torch.Size([10, 32, 32, 3])
torch.Size([10, 32, 32, 3])
(3072, 96, 3, 1)
(3072, 1, 96, 32)


Extract all layers in a model

In [ ]:
train_nodes, eval_nodes = get_graph_node_names(model)
print(train_nodes)
print(len(train_nodes))
# print(len(eval_nodes))

Return specific nodes of a model and create a partial architecture

In [ ]:
return_nodes = {
    'features.transition2.relu': 'layer1',
}
create_feature_extractor(model, return_nodes=return_nodes)

# **Move Files to and fro GDrive**

## Copy Experiment Results from Virtual Machine or VM (Contents) to a Google Drive Folder

In [ ]:
!cp -av '/content/Results/' '/content/GDrive/MyDrive/'

## Copy Documents from a Google Drive Folder to VM (Contents)

In [ ]:
!cp -av '/content/GDrive/MyDrive/Research/Kidney_AHN_Segmentation/Results' '/content'

## Remove non-empty Directory from the VM

In [ ]:
shutil.rmtree('/content/Results')

# **Infinite Loop**

In [ ]:
while True:
  pass